# SpringBoard Capstone2: Coronavirus Tweets Sentiment Analysis

    Data from Kaggle.com: https://www.kaggle.com/datatattle/covid-19-nlp-text-classification/metadata
## Modeled with tfidf

In [1]:
# !pip install wordcloud

In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Ling/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
# from wordcloud import WordCloud
import re
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
tweet = pd.read_csv('ProcessedTweets')

tweet.fillna('', inplace=True)
tweet.head()
tweet.isna().any()

In [29]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 

X = tweet['text'] 

y = tweet['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

# Initialize a TfidfVectorizer object: tfidf_vectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df = 10, max_df = 0.5)

# Transform the training data: tfidf_train 
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data: tfidf_test 
tfidf_test = tfidf_vectorizer.transform(X_test)

# Print the first 10 features
print('# of feature names', len(tfidf_vectorizer.get_feature_names()))
print(tfidf_vectorizer.get_feature_names()[:10])

# Print the first 5 vectors of the tfidf training data
print(tfidf_train.A[1])

# of feature names 5272
['ability', 'able', 'able get', 'about', 'about consumer', 'about coronavirus', 'about covid', 'about fod', 'about grocery', 'about people']
[0. 0. 0. ... 0. 0. 0.]


In [32]:
## Tried all these below FINALLY PICKED SVC

## Try NaiveBayes first
# Import the necessary modules
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB 
from sklearn.ensemble import RandomForestClassifier

# # using tfidf ## F1 0.44 fast to run
# NB_classifier = MultinomialNB()
# # Fit the classifier to the training data
# NB_classifier.fit(tfidf_train, y_train)
# # Create the predicted tags: pred
# pred = NB_classifier.predict(tfidf_test)
# # Calculate the accuracy score: score
# Score = metrics.accuracy_score(y_test, pred)
# print(Score)
# # Calculate the confusion matrix: cm
# CM = metrics.confusion_matrix(y_test, pred)
# print(CM)


# # Using Count_train
# nb_classifier = MultinomialNB()
# # Fit the classifier to the training data
# nb_classifier.fit(Count_train, y_train)
# # Create the predicted tags: pred
# pred = nb_classifier.predict(Count_test)
# # Calculate the accuracy score: score
# score = metrics.accuracy_score(y_test, pred)
# print(Score)
# # Calculate the confusion matrix: cm
# cm = metrics.confusion_matrix(y_test, pred)
# print(CM)

# # F1 = 0.49 fast to run
# BNBmodel = BernoulliNB()
# BNBmodel.fit(tfidf_train, y_train)
# y_pred = BNBmodel.predict(tfidf_test)
# print(classification_report(y_test, y_pred))
# print(metrics.accuracy_score(y_test, y_pred))

## F1 LinearSVC = 0.56
SVCmodel = LinearSVC()
SVCmodel.fit(tfidf_train, y_train)
y_pred = SVCmodel.predict(tfidf_test)
print(classification_report(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))
CM = metrics.confusion_matrix(y_test, y_pred)
print(CM)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs
                    precision    recall  f1-score   support

Extremely Negative       0.59      0.59      0.59      1342
Extremely Positive       0.63      0.61      0.62      1674
          Negative       0.50      0.47      0.49      2466
           Neutral       0.59      0.68      0.63      1939
          Positive       0.51      0.50      0.50      2869

          accuracy                           0.56     10290
         macro avg       0.57      0.57      0.57     10290
      weighted avg       0.55      0.56      0.55     10290

0.5557823129251701
[[ 793   12  396   68   73]
 [  15 1017   69   61  512]
 [ 411   79 1160  375  441]
 [  38   22  228 1325  326]
 [  90  481  464  410 1424]]


In [29]:
# hyper prameter tuning for SVC:
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'C': [0.1, 100], 'gamma': [1,0.001], 'kernel':['rbf','poly','sigmoid','linear'], 'degree':[0, 3]}
grid = GridSearchCV(SVC(),param_grid,scoring='accuracy', verbose=10)
grid.fit(tfidf_train,y_train)
 

# print best parameter after tuning
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.91 µs
Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5; 1/32] START C=0.1, degree=0, gamma=1, kernel=rbf.......................
[CV 1/5; 1/32] END .....C=0.1, degree=0, gamma=1, kernel=rbf; total time= 4.5min
[CV 2/5; 1/32] START C=0.1, degree=0, gamma=1, kernel=rbf.......................
[CV 2/5; 1/32] END .....C=0.1, degree=0, gamma=1, kernel=rbf; total time= 4.6min
[CV 3/5; 1/32] START C=0.1, degree=0, gamma=1, kernel=rbf.......................
[CV 3/5; 1/32] END .....C=0.1, degree=0, gamma=1, kernel=rbf; total time= 4.6min
[CV 4/5; 1/32] START C=0.1, degree=0, gamma=1, kernel=rbf.......................
[CV 4/5; 1/32] END .....C=0.1, degree=0, gamma=1, kernel=rbf; total time= 4.8min
[CV 5/5; 1/32] START C=0.1, degree=0, gamma=1, kernel=rbf.......................
[CV 5/5; 1/32] END .....C=0.1, degree=0, gamma=1, kernel=rbf; total time= 4.7min
[CV 1/5; 2/32] START C=0.1, degree=0, gamma=1, kernel=poly......

[CV 1/5; 11/32] END C=0.1, degree=3, gamma=1, kernel=sigmoid; total time= 3.9min
[CV 2/5; 11/32] START C=0.1, degree=3, gamma=1, kernel=sigmoid..................
[CV 2/5; 11/32] END C=0.1, degree=3, gamma=1, kernel=sigmoid; total time= 3.9min
[CV 3/5; 11/32] START C=0.1, degree=3, gamma=1, kernel=sigmoid..................
[CV 3/5; 11/32] END C=0.1, degree=3, gamma=1, kernel=sigmoid; total time= 3.8min
[CV 4/5; 11/32] START C=0.1, degree=3, gamma=1, kernel=sigmoid..................
[CV 4/5; 11/32] END C=0.1, degree=3, gamma=1, kernel=sigmoid; total time= 3.8min
[CV 5/5; 11/32] START C=0.1, degree=3, gamma=1, kernel=sigmoid..................
[CV 5/5; 11/32] END C=0.1, degree=3, gamma=1, kernel=sigmoid; total time= 3.8min
[CV 1/5; 12/32] START C=0.1, degree=3, gamma=1, kernel=linear...................
[CV 1/5; 12/32] END .C=0.1, degree=3, gamma=1, kernel=linear; total time= 4.1min
[CV 2/5; 12/32] START C=0.1, degree=3, gamma=1, kernel=linear...................
[CV 2/5; 12/32] END .C=0.1, 

[CV 2/5; 21/32] END C=100, degree=0, gamma=0.001, kernel=rbf; total time= 4.1min
[CV 3/5; 21/32] START C=100, degree=0, gamma=0.001, kernel=rbf..................
[CV 3/5; 21/32] END C=100, degree=0, gamma=0.001, kernel=rbf; total time= 4.0min
[CV 4/5; 21/32] START C=100, degree=0, gamma=0.001, kernel=rbf..................
[CV 4/5; 21/32] END C=100, degree=0, gamma=0.001, kernel=rbf; total time= 4.0min
[CV 5/5; 21/32] START C=100, degree=0, gamma=0.001, kernel=rbf..................
[CV 5/5; 21/32] END C=100, degree=0, gamma=0.001, kernel=rbf; total time= 4.1min
[CV 1/5; 22/32] START C=100, degree=0, gamma=0.001, kernel=poly.................
[CV 1/5; 22/32] END C=100, degree=0, gamma=0.001, kernel=poly; total time= 3.6min
[CV 2/5; 22/32] START C=100, degree=0, gamma=0.001, kernel=poly.................
[CV 2/5; 22/32] END C=100, degree=0, gamma=0.001, kernel=poly; total time= 3.7min
[CV 3/5; 22/32] START C=100, degree=0, gamma=0.001, kernel=poly.................
[CV 3/5; 22/32] END C=100,

[CV 3/5; 31/32] END C=100, degree=3, gamma=0.001, kernel=sigmoid; total time= 3.9min
[CV 4/5; 31/32] START C=100, degree=3, gamma=0.001, kernel=sigmoid..............
[CV 4/5; 31/32] END C=100, degree=3, gamma=0.001, kernel=sigmoid; total time= 3.9min
[CV 5/5; 31/32] START C=100, degree=3, gamma=0.001, kernel=sigmoid..............
[CV 5/5; 31/32] END C=100, degree=3, gamma=0.001, kernel=sigmoid; total time= 3.9min
[CV 1/5; 32/32] START C=100, degree=3, gamma=0.001, kernel=linear...............
[CV 1/5; 32/32] END C=100, degree=3, gamma=0.001, kernel=linear; total time= 5.8min
[CV 2/5; 32/32] START C=100, degree=3, gamma=0.001, kernel=linear...............
[CV 2/5; 32/32] END C=100, degree=3, gamma=0.001, kernel=linear; total time= 6.1min
[CV 3/5; 32/32] START C=100, degree=3, gamma=0.001, kernel=linear...............
[CV 3/5; 32/32] END C=100, degree=3, gamma=0.001, kernel=linear; total time= 6.0min
[CV 4/5; 32/32] START C=100, degree=3, gamma=0.001, kernel=linear...............
[CV 4/5

In [37]:
result=pd.DataFrame(grid.cv_results_)
result[['params','param_kernel', 'mean_test_score', 'rank_test_score']]

,params,param_kernel,mean_test_score,rank_test_score
0,"{'C': 0.1, 'degree': 0, 'gamma': 1, 'kernel': ...",rbf,0.282502,20
1,"{'C': 0.1, 'degree': 0, 'gamma': 1, 'kernel': ...",poly,0.277092,23
2,"{'C': 0.1, 'degree': 0, 'gamma': 1, 'kernel': ...",sigmoid,0.387631,18
3,"{'C': 0.1, 'degree': 0, 'gamma': 1, 'kernel': ...",linear,0.391356,12
4,"{'C': 0.1, 'degree': 0, 'gamma': 0.001, 'kerne...",rbf,0.277092,23
5,"{'C': 0.1, 'degree': 0, 'gamma': 0.001, 'kerne...",poly,0.277092,23
6,"{'C': 0.1, 'degree': 0, 'gamma': 0.001, 'kerne...",sigmoid,0.277092,23
7,"{'C': 0.1, 'degree': 0, 'gamma': 0.001, 'kerne...",linear,0.391356,12
8,"{'C': 0.1, 'degree': 3, 'gamma': 1, 'kernel': ...",rbf,0.282502,20
9,"{'C': 0.1, 'degree': 3, 'gamma': 1, 'kernel': ...",poly,0.277675,22


In [40]:
# F1 0.52, run time short
rf = RandomForestClassifier(random_state=42)
rf.fit(tfidf_train, y_train)
y_pred = rf.predict(tfidf_test)
print(classification_report(y_test, y_pred))
metrics.accuracy_score(y_test, y_pred)

                    precision    recall  f1-score   support

Extremely Negative       0.61      0.42      0.50      1342
Extremely Positive       0.64      0.46      0.53      1674
          Negative       0.48      0.48      0.48      2466
           Neutral       0.54      0.69      0.61      1939
          Positive       0.48      0.53      0.50      2869

          accuracy                           0.52     10290
         macro avg       0.55      0.52      0.52     10290
      weighted avg       0.53      0.52      0.52     10290



0.5232264334305151

In [42]:
# Searching for better hyperparameters
# Define Parameters
max_depth=[8, 64]
n_estimators = [64, 256, 512]
param_grid = {'max_depth': max_depth, 
              'n_estimators': n_estimators}

# Build the grid search
dfrst = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state = 1)
grid_RF = GridSearchCV(estimator=dfrst, param_grid=param_grid, cv = 5, scoring='accuracy', verbose=10)
grid_results = grid_RF.fit(tfidf_train,y_train)

# Summarize the results in a readable format
print("Best: {0}, using {1}".format(grid_results.cv_results_['mean_test_score'], grid_results.best_params_))
results_RF = pd.DataFrame(grid_results.cv_results_)
results_RF

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START max_depth=8, n_estimators=64................................
[CV 1/5; 1/6] END ..............max_depth=8, n_estimators=64; total time=   0.8s
[CV 2/5; 1/6] START max_depth=8, n_estimators=64................................
[CV 2/5; 1/6] END ..............max_depth=8, n_estimators=64; total time=   0.8s
[CV 3/5; 1/6] START max_depth=8, n_estimators=64................................
[CV 3/5; 1/6] END ..............max_depth=8, n_estimators=64; total time=   0.8s
[CV 4/5; 1/6] START max_depth=8, n_estimators=64................................
[CV 4/5; 1/6] END ..............max_depth=8, n_estimators=64; total time=   0.8s
[CV 5/5; 1/6] START max_depth=8, n_estimators=64................................
[CV 5/5; 1/6] END ..............max_depth=8, n_estimators=64; total time=   0.8s
[CV 1/5; 2/6] START max_depth=8, n_estimators=256...............................
[CV 1/5; 2/6] END .............max_depth=8, n_est

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.745753,0.012255,0.084325,0.002892,8,64,"{'max_depth': 8, 'n_estimators': 64}",0.301749,0.301263,0.304228,0.300664,0.301798,0.301941,0.001215,4
1,2.830138,0.149389,0.252849,0.012852,8,256,"{'max_depth': 8, 'n_estimators': 256}",0.295109,0.298348,0.295966,0.297748,0.296614,0.296757,0.001173,5
2,5.583545,0.419258,0.516784,0.073685,8,512,"{'max_depth': 8, 'n_estimators': 512}",0.294137,0.295109,0.294670,0.295480,0.294670,0.294813,0.000454,6
3,12.730127,2.498740,0.256962,0.093112,64,64,"{'max_depth': 64, 'n_estimators': 64}",0.502430,0.501944,0.493115,0.497813,0.498137,0.498688,0.003368,3
4,40.335381,2.319093,0.709170,0.038180,64,256,"{'max_depth': 64, 'n_estimators': 256}",0.509556,0.504535,0.501215,0.502835,0.497975,0.503223,0.003835,2
5,77.415099,2.169417,1.358657,0.049685,64,512,"{'max_depth': 64, 'n_estimators': 512}",0.511500,0.506317,0.504779,0.504131,0.500081,0.505361,0.003697,1
